In [1]:
import h5py
import os
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm

os.chdir('/mnt/sda3/yigedabuliu/lkq/brats-unet/')
os.getcwd()

'/mnt/sda3/yigedabuliu/lkq/brats-unet'

In [2]:
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from BraTS import *

In [4]:

class BraTS_SingleModal(Dataset):
    def __init__(self, data_path, file_path, modal, transform=None):
        with open(file_path, 'r') as f:
            self.paths = [os.path.join(data_path, x.strip()) for x in f.readlines()]
        self.transform = transform
        #
        self.modal_list = ['flair', 't1ce', 't1', 't2']
        self.modal = modal

    def __getitem__(self, item):
        h5f = h5py.File(self.paths[item], 'r')
        image = h5f['image'][:]
        label = h5f['label'][:]
        # [0,1,2,4] -> [0,1,2,3]
        label[label == 4] = 3
        # print(image.shape)
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)

        index = self.modal_list.index(self.modal)
        single_modal_img = sample['image'][index].unsqueeze(dim=0)

        return single_modal_img, sample['label']

    def __len__(self):
        return len(self.paths)

    def collate(self, batch):
        return [torch.cat(v) for v in zip(*batch)]

In [5]:
data_path = '/mnt/sda3/yigedabuliu/lkq/data/Task01_BrainTumour/h5_datasets'
train_txt = '/mnt/sda3/yigedabuliu/lkq/data/Task01_BrainTumour/train.txt'

patch_size = (160, 160, 128)

train_dataset = BraTS_SingleModal(data_path, train_txt, transform=transforms.Compose([
    RandomRotFlip(),
    RandomCrop(patch_size),
    GaussianNoise(p=0.1),
    ToTensor()
]), modal='flair')

In [6]:
img, label = train_dataset[0]

In [7]:
img.shape

torch.Size([1, 160, 160, 128])

In [8]:
modal = 't1ce'
modal_list = ['flair', 't1ce', 't1', 't2']

In [25]:
modal_list.index(modal)

1